# Практическое задание №14 по теме "Transfer learning".

In [1]:
! pip install datasets transformers

In [2]:
!pip install --upgrade accelerate

In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

In [4]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)
import warnings
warnings.filterwarnings('ignore')

C:\Users\rfk03\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\trainer_pt_utils.py:211: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: Optional[torch.device] = torch.device("cuda"),


In [5]:
model_name = 'ai-forever/rugpt3small_based_on_gpt2'

In [6]:
df = pd.read_json('C:\Study\dataset.jsonl', lines=True).set_index('id')

In [7]:
df.head(5)

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [8]:
def clear_text(text):
  clr_text = re.sub(r"<.*?>", " ",text).lower()
  clr_text = summary = re.sub(r"\s", " ", clr_text)
  return clr_text

In [9]:
df['clear_text'] = df['text'].apply(lambda x: clear_text(x))
df.head(5)

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...


In [10]:
data = df.loc[:,'clear_text']
data

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: clear_text, Length: 81497, dtype: object

In [11]:
def build_text_files(data_json, dest_path):
  f = open(dest_path, 'w', encoding="utf-8")
  data = ''
  for texts in data_json:
    summary = str(texts).strip()
    data += summary + " "
  f.write(data)

In [12]:
train, test = train_test_split(data, test_size = 0.2)

build_text_files(train, 'C:/Study/train_dataset.txt')
build_text_files(test, 'C:/Study/test_dataset.txt')

In [13]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/rugpt3small_based_on_gpt2')

train_path = 'C:/Study/train_dataset.txt'
test_path = 'C:/Study/test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
from transformers.data import data_collator
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
  train_dataset = TextDataset(
      tokenizer = tokenizer,
      file_path = train_path,
      block_size = 128)
  
  test_dataset = TextDataset(
      tokenizer = tokenizer,
      file_path = test_path,
      block_size = 128)
  
  data_collator = DataCollatorForLanguageModeling(
      tokenizer = tokenizer, mlm = False)
  
  return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

In [15]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_name)

In [16]:
training_args = TrainingArguments(
    'phrase',
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 1,
    learning_rate = 0.001,
    weight_decay = 0.01,
    save_strategy = 'no',
    report_to = 'none')

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: Numpy is not available

In [ ]:
def generate_text(prefix):
    tokens = tokenizer(prefix, ruturn_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]
    
    output = model.generate(
        **tokens,
        do_sample = False,
        max_length = size+50,
        early_stopping = True,
        length_penalty=2.0,
        repetiton_penalty=8.,
        temperature - 0.5,
        num_beams = 3,
        no_repeat_ngram_size=5
    )
    
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result
    

In [ ]:
print(generate_text('На фоне тьмы ввысь белым бил фонтан'))